In [36]:

from langchain_community.llms import Ollama

MODEL = "llama3"

model = Ollama(model=MODEL)

In [65]:
links = [
    'https://imsdb.com/transcripts/South-Park-Cartman-gets-an-anal-probe.html',
    'https://imsdb.com/transcripts/South-Park-Weight-gain-4000.html',
    # 'https://imsdb.com/transcripts/South-Park-Volcano.html',
    # 'https://imsdb.com/transcripts/South-Park-Big-Gay-Al\'s-big-gay-boatride.html',
    # 'https://imsdb.com/transcripts/South-Park-An-elephant-makes-love-to-a-pig.html',
    # 'https://imsdb.com/transcripts/South-Park-Death.html',
    # 'https://imsdb.com/transcripts/South-Park-Pinkeye.html',
    # 'https://imsdb.com/transcripts/South-Park-Damien.html',
    # 'https://imsdb.com/transcripts/South-Park-Starvin-Marvin.html',
    # 'https://imsdb.com/transcripts/South-Park-Mr-Hankey-the-Christmas-Poo.html',
    # 'https://imsdb.com/transcripts/South-Park-Tom\'s-rhinoplasty.html',
    # 'https://imsdb.com/transcripts/South-Park-Mecha-Streisand.html',
    # 'https://imsdb.com/transcripts/South-Park-Cartman\'s-mom-is-a-dirty-slut.html'
]

In [39]:
import nest_asyncio

nest_asyncio.apply()

In [68]:
%pip install html2text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 1.1 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for html2text: filename=html2text-2024.2.26-py3-none-any.whl size=33111 sha256=708e30c5b4f6f03cd77a168c58d36e10c571ae6269f3352bd74d02129a3e9dfa
  Stored in directory: /Users/serjou/Library/Caches/pip/wheels/2b/01/23/578505d65e2a97d78bf1fe3fc8256ecf37572dc1df598b0eaf
Successfully built html2text
Note: you may need to restart the kernel to use updated packages.


In [91]:
from langchain_community.document_loaders import AsyncChromiumLoader
from langchain_community.document_transformers import Html2TextTransformer, BeautifulSoupTransformer
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = AsyncChromiumLoader(links)
docs = loader.load()

bs_transformer = BeautifulSoupTransformer()
docs = bs_transformer.transform_documents(documents=docs, tags_to_extract=["td"])

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs = text_splitter.transform_documents(docs)

len(docs)
# bodies
# html2text = Html2TextTransformer()
# docs_transformed = html2text.transform_documents(docs)
# docs_transformed



74

In [182]:
from langchain_community.document_loaders import TextLoader

from langchain.vectorstores.utils import filter_complex_metadata

# text_loader1 = TextLoader('transcripts/Episode 101.txt')
# docs = text_loader1.load()



text_loader1 = TextLoader('transcripts/Episode 102.txt')
docs = text_loader1.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=100)
docs = text_splitter.transform_documents(docs)
# docs = filter_complex_metadata(docs)

# docs = docs[:10]
len(docs)

40

In [172]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores.utils import filter_complex_metadata


docs = PyPDFLoader(file_path='pdfs/e102.pdf').load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=100)
chunks = text_splitter.split_documents(docs)
chunks = filter_complex_metadata(chunks)



In [183]:
from langchain_community.embeddings import OllamaEmbeddings, FastEmbedEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_chroma import Chroma


# Chroma().delete_collection()
vector_store = Chroma.from_documents(documents=docs, embedding=FastEmbedEmbeddings())

# embeddings = FastEmbedEmbeddings()
# vectorstore = Chroma.from_documents(docs, embeddings)

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 43509.38it/s]


In [184]:
retriever = vector_store.as_retriever(
    # search_type="similarity_score_threshold",
    # search_kwargs={
    #     "k": 3,
    #     "score_threshold": 1,
    # }
    )
# retriever.invoke('Why does Cartman hates rainbows?')
retriever.invoke('Why does Cartman hates rainbows?')

[Document(page_content="Hey Stan, did you see that rainbow \n                         this morning?\n                                     STAN\n                         Yeah, it was huge!\n                                     CARTMAN\n                         Oh, I hate those things.\n               The kids all look at Cartman puzzlingly.\n                                     KYLE\n                         Nobody HATES rainbows.\n                                     STAN\n                         Yeah, what's there to HATE about \n                         rainbows?!\n                                     CARTMAN\n                         Well, you know... You'll just be \n                         sitting there minding your own \n                         business, and they'll come marching \n                         in and crawl up your leg and start \n                         biting the inside of your ass -\n               Cartman makes gnashing sounds with his teeth.\n                

In [176]:
retriever.invoke('How does cartman get red eye?')

[Document(page_content="to the store now then.\n               Cartman's mom pulls her purse on and heads out the door.\n               INT. MR. GARRISON'S BEDROOM - NIGHT\n               Garrison is lying in bed, tossing and turning from a terrible \n               nightmare. We see whispy images floating through Garrison's \n               thoughts -\n               Finally, Garrison awakes, and jolts up to a sitting position.\n                                     MR. GARRISON\n                         NO, NO!!!!\n               Garrison wipes sweat from his brow, realizing it was all a \n               dream. Then he spots Mr. Hat in his little chair.\n                                     MR. HAT\n                         Kill her.\n                                     MR. GARRISON\n                         No, Mr. Hat! I won't do it!\n               Mr. Hat's eyes begin to glow red. Mr. Hat's head starts \n               spinning around and around.\n                                

In [ ]:
('Rainbows are little arches of color')

In [188]:
from langchain.prompts import PromptTemplate
# template = """
#     Answer the question based on the context below. If you can't 
#     answer the question, reply "I don't know".

#     Context: {context}

#     Question: {question}
#     """

template = """
    Answer the question based on the context below. If you can't 
    answer the question, reply with context.

    Context: {context}

    Question: {question}
    """

prompt = PromptTemplate.from_template(template)

In [189]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


parser = StrOutputParser()
chain = (
            {"context": vector_store.as_retriever(), "question": RunnablePassthrough()}
            | prompt
            | model
            | parser
    )

chain2 = (
    {"context": vector_store.as_retriever(), "question": RunnablePassthrough()}
            | prompt
)

In [141]:
chain.invoke("What was in Cartman's ass?")

"According to the transcript, Kenny's body is seen flying across the stage and landing upside down in a large barrel of water after being hit by the bullet. The camera then cuts away before showing what's inside the barrel, so we don't get a direct answer to this question.\n\nHowever, based on the context and the reactions of the other characters, it can be inferred that Kenny's body was inside the barrel."

In [186]:
chain2.invoke("Why does Cartman hates rainbows?")

StringPromptValue(text='\n    Answer the question based on the context below. If you can\'t \n    answer the question, reply "I don\'t know".\n\n    Context: [Document(page_content="Hey Stan, did you see that rainbow \\n                         this morning?\\n                                     STAN\\n                         Yeah, it was huge!\\n                                     CARTMAN\\n                         Oh, I hate those things.\\n               The kids all look at Cartman puzzlingly.\\n                                     KYLE\\n                         Nobody HATES rainbows.\\n                                     STAN\\n                         Yeah, what\'s there to HATE about \\n                         rainbows?!\\n                                     CARTMAN\\n                         Well, you know... You\'ll just be \\n                         sitting there minding your own \\n                         business, and they\'ll come marching \\n                     

In [190]:
chain.invoke("Why does Cartman hates rainbows?")

'Based on the context, it seems that Cartman hates rainbows because he believes they will "come marching in and crawl up your leg and start biting the inside of your ass". This is a bizarre and disturbing reason for someone to hate something as beautiful and harmless as rainbows!'

In [100]:
chain.invoke("How does cartman feel about rainbow?")

"I'm not seeing any context that would allow me to answer this question based on the provided HTML code. The code appears to be a web page template with some script tags and links, but it doesn't contain any information about Cartman's feelings towards rainbows or anything related to South Park.If you meant to provide more context or clarify the question, I'd be happy to help!"

In [122]:
chain.invoke("How does cartman get red eye?")

'I\'m happy to help! However, I need some more context to answer your question accurately. Since the webpage you provided is a transcript database for movie and TV show scripts, it seems that Cartman getting an anal probe might be related to the South Park episode "Cartman Gets an Anal Probe" (Season 1, Episode 7). \n\nIn this episode, Cartman gets abducted by aliens who perform an anal probe on him. This is a memorable and humorous moment in the series. If you have any further questions or clarification regarding red eye specifically, please let me know!'

In [129]:
chain.invoke('in which episode cartman gets an anal probe?')

'Based on the context provided, it appears that the question is asking about a specific episode of the TV show "South Park" where Cartman gets an anal probe.\n\nAccording to my analysis, the correct answer would be:\n\nThe episode where Cartman gets an anal probe is "Trapped in the Closet" (Season 8, Episode 11).'

In [124]:
retriever.invoke('Who won the national \'Save Our Fragile Planet\'?')

[Document(page_content="cigarette. CHEF He needs to run his ass around the block a few times. An arm drapes over Chef. PULL BACK to reveal that Kathie Lee is lying in bed naked next to Chef. KATHIE LEE GIFFORD Mmmm, how about a little more of that good loving, Chef? CHEF Damn woman, I just gave you sweet lovin' 5 minutes ago. You tryin' to kill me? FADE OUT: THE END", metadata={'source': 'transcripts/Episode 102.txt'}),
 Document(page_content='Mooo. Moooo. Mooo. SUBTITLES: "Greetings, Cows of Earth. We come in peace." The cows all look at each other. Finally, one cow makes a noise. COW Moo? SUBTITLES: "Really?" Meanwhile, Kyle and the kids are still trying to get Ike to jump down. KYLE Come on, Ike! I promise I\'ll be nice to you from now on! IKE Don kick da bebe! The visitors continue to moo at the cows. VISITOR Moo. Moo. Moo. SUBTITLES: "We have experimented with all the beings of Earth, and have learned that you are the most intelligent and wise." CARTMAN What the hell are they talk

In [136]:
embeddings.embed_query('Who won the national \'Save Our Fragile Planet\'?')

[-2.948016405105591,
 -2.7814834117889404,
 0.43418440222740173,
 1.1867581605911255,
 2.150810480117798,
 -1.338158369064331,
 0.5854767560958862,
 1.9074195623397827,
 -4.601661682128906,
 -1.1167775392532349,
 -0.9460458159446716,
 2.9803307056427,
 -4.329325199127197,
 -0.911036491394043,
 -0.6565566658973694,
 -1.155159592628479,
 -2.9511427879333496,
 -2.884497880935669,
 -2.97922945022583,
 2.965806007385254,
 -1.1082967519760132,
 -1.1236718893051147,
 1.4756097793579102,
 -0.10116676241159439,
 1.4793001413345337,
 1.7951291799545288,
 -1.283229112625122,
 -0.5747070908546448,
 0.566578209400177,
 -2.549786329269409,
 -0.760069727897644,
 3.2790722846984863,
 -1.0000056028366089,
 3.618237018585205,
 4.564723968505859,
 -3.0242919921875,
 -4.653653144836426,
 -0.6689355373382568,
 4.265432357788086,
 2.84482741355896,
 -0.5203937292098999,
 0.5529276728630066,
 1.5392608642578125,
 0.12481538951396942,
 -0.753002941608429,
 1.9722623825073242,
 0.4404239356517792,
 2.356123924